In [1]:
 '''
Author information:
Joey R. Muffoletto
University of Texas at Austin
Autonomous Systems Group
jrmuff@utexas.edu
'''

'\nAuthor information:\nJoey R. Muffoletto\nUniversity of Texas at Austin\nAutonomous Systems Group\njrmuff@utexas.edu\n'

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [16]:
NUM_VERTIHUBS = 10 #dependent on trip_data
# PORTS_PER_VERTIHUB = 

In [17]:
'''
Trip data formatting
'''
trip_data = pd.read_csv('data/Realtime/OpsLimits/trips.csv')
trip_data.drop(trip_data[trip_data.Origin == trip_data.Destination].index, inplace=True) #drop trips where the origin and destination are the same vertiport, reduced from 4801 trips to 1692 with 10 vertiports


In [18]:
'''
input statistics
'''
trips_per_vertiport = [0 for i in range(NUM_VERTIPORTS)]
# for trip in trips:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-18-56206bdf289a>, line 5)